In [1]:
import os
import glob
import shutil
import pandas as pd
from PIL import Image

In [2]:
partner_list_from_Images = glob.glob('Images/*')
partner_list_from_Images = sorted(partner_list_from_Images)
partner_list = [s.replace('Images/','') for s in partner_list_from_Images]

In [3]:
idx_all = 0
df = pd.DataFrame([],index=['museum','url','detail'])
for partner_name in partner_list:
    image_idx_list = glob.glob('Images/'+partner_name+'/*')
    image_idx_list = [int(s.replace('Images/'+partner_name+'/','')) for s in image_idx_list]
    image_idx_list = sorted(image_idx_list)

    for image_idx in image_idx_list:
        shutil.copyfile('Images/' + partner_name + '/' + str(image_idx), 'GAC_Dataset/Images/' + str(idx_all)) # Copy Image
        # Convert JPEG
        pil_img = Image.open('GAC_Dataset/Images/' + str(idx_all), 'r')
        rgb_img = pil_img.convert('RGB')
        rgb_img.save('GAC_Dataset/Images/' + str(idx_all) + '.jpg', 'JPEG')
        os.remove('GAC_Dataset/Images/' + str(idx_all))
        # Save Detail
        detail = pd.read_csv('Details/' + partner_name + '/' + str(image_idx) +'.csv',index_col=0).values.flatten()
        url = 'https://artsandculture.google.com/partner/'
        museum_url = url+partner_name
        df[str(idx_all)] = [partner_name,museum_url,detail]

        idx_all += 1

In [7]:
df.T.to_csv('GAC_Dataset/details.csv')

## Crop & Resize Images

In [ ]:
LOAD_DIR = 'GAC_Dataset/Images/'
#SAVE_DIR = LOAD_DIR
SAVE_DIR = 'GAC_Dataset/Images_Edited/'

In [ ]:
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

In [ ]:
for i in range(len(glob.glob(LOAD_DIR+'*'))):
    img = Image.open(LOAD_DIR + str(i) + '.jpg')
    w,h = img.size

    if w > h:
        if w/h > 1.2:
            img = crop_max_square(img)
        img_new = img.resize((224, 224))
    else:
        if h/w > 1.2:
            img = crop_max_square(img)
        img_new = img.resize((224, 224))

    img_new.save(SAVE_DIR + str(i) + '.jpg')